In [ ]:
# automatically reload the package when it is modified
%load_ext autoreload
%autoreload 2

In [ ]:

from mtr.config import cfg, cfg_from_yaml_file
from mtr.datasets.waymo.waymo_dataset_bc import WaymoDatasetBC
from tqdm import tqdm
import numpy as np

cfg_file = 'tools/cfgs/waymo/bc+10_percent_data.yaml'
cfg = cfg_from_yaml_file(cfg_file, cfg)
cfg.DATA_CONFIG.WITHOUT_HDMAP = True
dataset = WaymoDatasetBC(cfg.DATA_CONFIG, training=True)
num_data = len(dataset)


In [ ]:
for i in tqdm(range(num_data)):
    scene_id, info = dataset.load_info(i)
    track_index_to_predict = np.array(info['tracks_to_predict']['track_index'])
    track_infos = info['track_infos']
    obj_trajs_raw = track_infos['trajs']

    # obj_trajs_raw[...,6] = np.arctan2(np.sin(obj_trajs_raw[...,6]), np.cos(obj_trajs_raw[...,6]))

    obj_trajs_pred = obj_trajs_raw[track_index_to_predict]

    delta = obj_trajs_pred[:, 1:, [0,1,6]] - obj_trajs_pred[:, :-1, [0,1,6]]
    delta[..., -1] = np.arctan2(np.sin(delta[..., -1]), np.cos(delta[..., -1]))
    delta_valid = np.logical_and(obj_trajs_pred[:, 1:, -1], obj_trajs_pred[:, :-1, -1])
    # delta[delta_valid].shape
    max = np.max(delta[delta_valid], axis=0)

    if np.any(max[:2]>5):
        break

In [ ]:
import matplotlib.pyplot as plt
print(max)
a = 0
mask = obj_trajs_pred[a,:,-1] == 1

temp_delta_mask = np.logical_and(obj_trajs_pred[a, 1:, -1], obj_trajs_pred[a, :-1, -1])
temp_delta_valid = delta[a, temp_delta_mask]
print(temp_delta_valid.max(0))
plt.plot(obj_trajs_pred[a, mask, 0], obj_trajs_pred[a, mask, 1])
_ = plt.axis('equal')

In [ ]:
def calc(n, running_sum, running_sum_square):
    mean  = running_sum / n
    var = running_sum_square / n - mean**2
    std = var**0.5
    
    print('mean: ', mean)
    print('var: ', var)
    print('std: ', std)
    return mean, var, std

In [96]:
running_sum = 0
running_sum_square = 0
n = 0
max = 0

for i in tqdm(range(num_data)):
    scene_id, info = dataset.load_info(i)
    for t in range(10, 90, 10):
        data = dataset.extract_scene_data(scene_id, info, t)
        if data is None:
            continue
        gt = data['center_gt'][...,:3]
        running_sum += gt.sum(axis=0)
        running_sum_square += (gt**2).sum(axis=0)
        n += gt.shape[0]
        batch_max = np.abs(gt).max(axis=0)
        
        max = np.maximum(batch_max, max)
    if i%500 == 0:    
        print(f'itr: {i}, max: {max}')
        mean, var, std = calc(n, running_sum, running_sum_square)
    
mean, var, std = calc(n, running_sum, running_sum_square)


  0%|          | 24/49065 [00:00<06:53, 118.50it/s]

itr: 0, max: [1.5082357  0.06839956 0.04676199]
mean:  [ 0.6032526  -0.00936969 -0.00712883]
var:  [2.3693201e-01 4.4795254e-04 1.9416364e-04]
std:  [0.48675662 0.02116489 0.01393426]


  1%|          | 510/49065 [00:04<05:37, 143.87it/s]

itr: 500, max: [3.7786407 0.3006466 3.1325724]
mean:  [0.6409415  0.00110349 0.00119   ]
var:  [0.27809155 0.00046887 0.00378586]
std:  [0.52734387 0.02165329 0.06152934]


  2%|▏         | 1025/49065 [00:10<07:12, 111.11it/s]

...

itr: 48500, max: [4.8012466 2.336453  3.1415606]
mean:  [0.65435887 0.00128618 0.00069326]
var:  [0.29664635 0.00051253 0.00324353]
std:  [0.54465251 0.0226391  0.05695203]


100%|█████████▉| 49010/49065 [09:03<00:00, 86.68it/s] 

itr: 49000, max: [4.8012466 2.336453  3.1415606]
mean:  [0.6541934  0.00128473 0.00069162]
var:  [0.29644853 0.00051418 0.003253  ]
std:  [0.54447087 0.02267551 0.05703504]


100%|██████████| 49065/49065 [09:04<00:00, 90.11it/s] 

mean:  [0.65429819 0.00128331 0.0006909 ]
var:  [0.29647788 0.00051421 0.00324952]
std:  [0.54449782 0.02267612 0.05700458]
